In [40]:
!sudo apt install texlive-latex-extra

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
texlive-latex-extra is already the newest version (2021.20220204-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [41]:
!sudo apt install texlive-fonts-recommended

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
texlive-fonts-recommended is already the newest version (2021.20220204-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [42]:
!sudo apt-get install texlive-latex-base

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
texlive-latex-base is already the newest version (2021.20220204-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [43]:
!pip install PyLaTeX

In [44]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [45]:
import xlrd
import os
import pandas as pd
import numpy as np
import urllib.request
import cv2
from google.oauth2 import service_account
from googleapiclient.discovery import build
from pylatex import Document, Section, Figure, NoEscape, Command
from pylatex.utils import NoEscape
import gspread as gs
import pandas as pd
import requests
import re
from PIL import Image
from io import BytesIO
import json
import google.auth.transport.requests
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

In [46]:
df = pd.read_excel("/content/gdrive/MyDrive/SSY_SEVA_24/IDY -2024 (Responses).xlsx")

In [ ]:
# df.set_index(['Country','State'], inplace=True)

In [47]:
filtered_df = df[df['Country'] == 'India']

In [48]:
sorted_df = filtered_df.sort_values(by=['State'])

In [49]:
sorted_df.columns

Index(['Unnamed: 0', 'Timestamp', 'Country', 'State', 'Organiser's Name(s)',
       'Organiser's Email ', 'Organiser's Mobile Number',
       'Organiser's Teacher/Volunteer Code',
       'Venue/Institute Name (Name of the place where event was organised)\n',
       'Type of Institute', 'Name of the Institute Contact Person',
       'Telephone Number of the Institute Contact Person',
       'City/Town Organised in\n', 'No. of Participants\n',
       'Date of the event', 'Upload Max 2 Pictures of the Event',
       'Upload a video of the Event', 'Chief Guest Name ',
       'Name of the Country', 'Organiser's Name(s).', 'Organiser's Email.',
       'Organiser's Mobile Number.', 'Organiser's Teacher/Volunteer Code.',
       'Venue/Institute Name (Name of the place where event was organised).\n',
       'Type of Institute.', 'Name of the Institute Contact Person.',
       'Telephone Number of the Institute Contact Person.',
       'City/Town Organised in.\n', 'Number of Participants',
     

In [50]:
def convert_drive_link(shared_link):
    # Check if the link contains 'open?id=' format
    if 'open?id=' in shared_link:
        file_id = shared_link.split('open?id=')[-1]
    elif '/d/' in shared_link:
        # Handle if the link is already in the direct access format
        file_id = shared_link.split('/d/')[-1].split('/')[0]
    else:
        raise ValueError("The provided link is not a valid Google Drive link.")

    # Construct the direct access link
    direct_access_link = f"https://drive.google.com/file/d/{file_id}/view"
    return direct_access_link

In [51]:
# Define the scope
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

# Path to your service account key file
SERVICE_ACCOUNT_FILE = '/content/gdrive/MyDrive/SSY_SEVA_24/ssy-seva-24.json'

# Extract the file ID from the provided Google Drive link
def extract_file_id(drive_link):
    match = re.search(r'/d/([a-zA-Z0-9_-]+)', drive_link)
    if match:
        return match.group(1)
    else:
        raise ValueError("Invalid Google Drive link")

# Fetch the file name using Google Drive API
def get_file_name(file_id):
    # Authenticate with the service account
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    # Build the Google Drive service
    service = build('drive', 'v3', credentials=credentials)

    # Get the file metadata
    file = service.files().get(fileId=file_id, fields='name').execute()

    return file.get('name')

In [52]:
# Create a new LaTeX document
doc = Document(documentclass='article')

In [53]:
for state, group in sorted_df.groupby('State'):
    #print(state,group)
    with doc.create(Section(state)):
        #to add column with number of participants
        # total_participants = group['No_of_participants'].sum()
        # print("doc create")
        total_participants = 1000 #temp
        num_venues = group['State'].nunique()
        doc.append(NoEscape(f"Outreach: {total_participants}\n"))
        doc.append(NoEscape(f"\nVenue: {num_venues}\n"))

        for index, row in group.iterrows():
          image_path = row['Upload Max 2 Pictures of the Event'].split(',')
          print("image_path",image_path)
          print("Hello why are you not working")
          for images in image_path:
            print(images)
            drive_link = convert_drive_link(images)
            try:
              file_id = extract_file_id(drive_link)
              print("file id:",file_id)
              file_name = get_file_name(file_id)
              print("file name:",file_name)
              print(f'The file name is: {file_name}')
            except Exception as e:
              print(f'An error occurred: {str(e)}')
              continue #change
            # Check if the file is an image
            # if file_name.endswith('.jpg') or file_name.endswith('.jpeg') or file_name.endswith('.png'):
            file_path = r'/content/gdrive/MyDrive/SSY_SEVA_24/National Pics/'+file_name
            # Add the image
            with doc.create(Figure(position='h!')) as fig:
              print("Hello image writing")
              fig.add_image(file_path, width=NoEscape(r'0.8\textwidth'))
              # fig.add_image(file_path, width='100px')
              venue_name = 'Venue/Institute Name (Name of the place where event was organised)\n'
              city_name = 'City/Town Organised in\n'
              fig.add_caption(f"{row[venue_name]},{row[city_name]}")
              # fig.append(NoEscape(r'\centering'))
              fig.append(Command('vspace','0.3cm'))

image_path ['https://drive.google.com/open?id=1ujtMCh00tElzpVjdyFhBFQh2YhGBfSwm', ' https://drive.google.com/open?id=1JR_kNKwOfzAIyLu65tLGElFbfDnk2y3-']
Hello why are you not working
https://drive.google.com/open?id=1ujtMCh00tElzpVjdyFhBFQh2YhGBfSwm
file id: 1ujtMCh00tElzpVjdyFhBFQh2YhGBfSwm
file name: IMG-20240616-WA0038 - Banashree Saikia.jpg
The file name is: IMG-20240616-WA0038 - Banashree Saikia.jpg
Hello image writing
 https://drive.google.com/open?id=1JR_kNKwOfzAIyLu65tLGElFbfDnk2y3-
file id: 1JR_kNKwOfzAIyLu65tLGElFbfDnk2y3-
file name: IMG-20240616-WA0036 - Banashree Saikia.jpg
The file name is: IMG-20240616-WA0036 - Banashree Saikia.jpg
Hello image writing
image_path ['https://drive.google.com/open?id=1V2ncuLUasw13JkJZyEZqggbAKbQX4drH', ' https://drive.google.com/open?id=1saiMCIBTXSrcQd8LJRbB8HjEvhwVOZzW', ' https://drive.google.com/open?id=1FCuhLH0u_eZm6r-UnmGozKC3-ZHRjAFK']
Hello why are you not working
https://drive.google.com/open?id=1V2ncuLUasw13JkJZyEZqggbAKbQX4drH
file 

In [56]:
doc.generate_pdf(r'/content/gdrive/MyDrive/SSY_SEVA_24/idk', clean_tex=False)

SyntaxError: unterminated string literal (detected at line 1) (<ipython-input-56-a227252ca05e>, line 1)